# Curso de aprendizaje automatizado
PCIC, UNAM

Machine Learning

Rodrigo S. Cortez Madrigal

<img src="https://pcic.posgrado.unam.mx/wp-content/uploads/Ciencia-e-Ingenieria-de-la-Computacion_color.png" alt="Logo PCIC" width="128" />  

### Tarea 1: Clasificador bayesiano ingenuo

- Divide aleatoriamente el conjunto de datos de cáncer de seno de Wisconsin en un subconjunto de
entrenamiento con el 60 % de los datos, un subconjunto de validación con el 20 % y un subconjunto
de prueba con el 20 % restante usando 0 como semilla para tu generador de números aleatorios.

- Entrena distintos clasificadores de tumores de seno y evalúalos tanto con el subconjunto de
entrenamiento como con el subconjunto de validación y discute su desempeño. 

- Existen 16 registros en el conjunto de datos con un atributo no especificado. Investiga estrategias para rellenar los datos
faltantes, utiliza las que consideres más adecuadas para este problema y discute el impacto en el
desempeño del clasificador. Reporta el porcentaje de predicciones correctas en el subconjunto de
prueba para el clasificador con mejor rendimiento en el subconjunto de validación.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.naive_bayes as nb
from sklearn.metrics import accuracy_score

import plotly.express as px

Este conjunto de datos contiene 699 registros de tumores de seno, de los cuales 458 son benignos y
241 son malignos.

Cada registro consta de los siguientes atributos

| Número | Atributo                               | Valores          |
|--------|----------------------------------------|------------------|
| 1      | Código de la muestra                  | ID               |
| 2      | Grosor del tumor                      | 1-10             |
| 3      | Uniformidad del tamaño de la célula   | 1-10             |
| 4      | Uniformidad de la forma de la célula  | 1-10             |
| 5      | Adhesión marginal                     | 1-10             |
| 6      | Tamaño de célula epitelial            | 1-10             |
| 7      | Núcleos desnudos                      | 1-10             |
| 8      | Cromatina blanda                      | 1-10             |
| 9      | Nucléolos normales                    | 1-10             |
| 10     | Mitosis de células                    | 1-10             |
| 11     | Clase                                 | 2 para benigno, 4 para maligno |

In [2]:
# Descargar el conjunto de datos
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"
data = pd.read_csv(url, header=None, index_col=0, sep=',', na_values='?')
data.columns = ["clump_thickness", "uniformity_of_cell_size", "uniformity_of_cell_shape", "marginal_adhesion", "single_epithelial_cell_size", "bare_nuclei", "bland_chromatin", "normal_nucleoli", "mitoses", "class"]


In [3]:
# Estrategias para manejar los datos faltantes

# 1. Eliminar las filas con datos faltantes
data1 = data.dropna()

# 2. Reemplazar los datos faltantes con la media de la columna
data2 = data.fillna(data.mean())

# 3. Reemplazar los datos faltantes con la moda de la columna
data3 = data.fillna(data.mode().iloc[0])

# 4. Reemplazar los datos faltantes con la mediana de la columna
data4 = data.fillna(data.median())

# 5. Reemplazar los datos faltantes con un valor constante
data5 = data.fillna(0)

macrodata = [data1, data2, data3, data4, data5] #(Severance Reference)
stategies = ["Eliminar filas", "Reemplazar con la media", "Reemplazar con la moda", "Reemplazar con la mediana", "Reemplazar con un valor constante"]

In [4]:
total_faltantes = len(data) - len(data.dropna())
print("Total de datos faltantes:", total_faltantes)
porcentaje_faltantes = total_faltantes/len(data)*100
print("Porcentaje de datos faltantes:", porcentaje_faltantes, "%")

x = [porcentaje_faltantes, 100-porcentaje_faltantes]
fig = px.pie(values=x, names=["Datos faltantes", "Datos no faltantes"], title="Porcentaje de datos faltantes")
fig.show()

Total de datos faltantes: 16
Porcentaje de datos faltantes: 2.28898426323319 %


In [5]:
data

,clump_thickness,uniformity_of_cell_size,uniformity_of_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,,,,,,,,,,
1000025,5,1,1,1,2,1.0,3,1,1,2
1002945,5,4,4,5,7,10.0,3,2,1,2
1015425,3,1,1,1,2,2.0,3,1,1,2
1016277,6,8,8,1,3,4.0,3,7,1,2
1017023,4,1,1,3,2,1.0,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...
776715,3,1,1,1,3,2.0,1,1,1,2
841769,2,1,1,1,2,1.0,1,1,1,2
888820,5,10,10,3,7,3.0,8,10,2,4


Reporta el porcentaje de correos que están etiquetados como spam y como no spam en el
conjunto de datos.

In [6]:
target = data1['class'].value_counts()

fig = px.pie(values=target, title='Distribución de clases', names={2: 'Benigno', 4: 'Maligno'})
fig.show()

print("Distribución de clases")
print(target)


Distribución de clases
class
2    444
4    239
Name: count, dtype: int64


Divide aleatoriamente el conjunto de datos en el 60 % para entrenamiento, el 20 % para validación y el 20 % restante para prueba usando 0 como semilla para tu generador de números aleatorios.

### Gaussian Naive Bayes

In [7]:
# Entrenar clasificadores bayesianos ingenuos

grid = []

for model in [nb.GaussianNB(), nb.MultinomialNB(), nb.BernoulliNB()]:
    for i, data in enumerate(macrodata):
        
        # Separar características y etiquetas
        X = data.iloc[:, :-1]
        y = data.iloc[:, -1]

        # Dividir el conjunto de datos en entrenamiento (60%) y temporal (40%)
        X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)

        # Dividir el conjunto temporal en validación (50% de 40% = 20%) y prueba (50% de 40% = 20%)
        X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

        #print("Strategy:", stategies[i])

        #print("Hay NaN en X_train:", X_train.isnull().values.any())
        #print("Hay NaN en X_val:", X_val.isnull().values.any())
        #print("Hay NaN en X_test:", X_test.isnull().values.any())

        model.fit(X_train, y_train)

        # Predecir y reportar el porcentaje de predicciones correctas en entrenamiento y validación
        y_train_pred_gnb = model.predict(X_train)
        y_val_pred_gnb = model.predict(X_val)
        y_test_pred_gnb = model.predict(X_test)

        train_accuracy_gnb = accuracy_score(y_train, y_train_pred_gnb) * 100
        val_accuracy_gnb = accuracy_score(y_val, y_val_pred_gnb) * 100
        test_accuracy_gnb = accuracy_score(y_test, y_test_pred_gnb) * 100

        #print(f"Modelo: {model.__class__.__name__}")
        #print(f"Estretegia: {stategies[i]}")
        #print(f"Exactitud en entrenamiento: {train_accuracy_gnb:.2f}%")
        #print(f"Exactitud en validación: {val_accuracy_gnb:.2f}%")

        grid.append([model.__class__.__name__, stategies[i], train_accuracy_gnb, val_accuracy_gnb, test_accuracy_gnb])

grid = pd.DataFrame(grid, columns=['Modelo', 'Estrategia', 'Exactitud en entrenamiento', 'Exactitud en validación', 'Exactitud en prueba'])


In [8]:
grid

,Modelo,Estrategia,Exactitud en entrenamiento,Exactitud en validación,Exactitud en prueba
0,GaussianNB,Eliminar filas,97.066015,94.160584,96.350365
1,GaussianNB,Reemplazar con la media,96.897375,92.142857,97.142857
2,GaussianNB,Reemplazar con la moda,96.897375,92.142857,96.428571
3,GaussianNB,Reemplazar con la mediana,96.897375,92.142857,96.428571
4,GaussianNB,Reemplazar con un valor constante,96.897375,92.142857,96.428571
5,MultinomialNB,Eliminar filas,91.931540,86.131387,89.781022
6,MultinomialNB,Reemplazar con la media,89.498807,87.857143,90.000000
7,MultinomialNB,Reemplazar con la moda,91.408115,87.857143,90.000000
8,MultinomialNB,Reemplazar con la mediana,91.408115,87.857143,90.000000
9,MultinomialNB,Reemplazar con un valor constante,91.646778,87.857143,90.000000


In [9]:
# Plotear la exactitud en entrenamiento y validación

fig = px.bar(grid, x='Modelo', y='Exactitud en validación', color='Estrategia', barmode='group', title='Exactitud en entrenamiento')
fig.show()

In [10]:
fig = px.bar(grid, x='Modelo', y='Exactitud en prueba', color='Estrategia', barmode='group', title='Exactitud en entrenamiento')
fig.show()

In [11]:
# Seleccionar la mejor estrategia para cada clasificador por validación

best_strategies = grid.loc[grid.groupby('Modelo')['Exactitud en validación'].idxmax()]
best_strategies

,Modelo,Estrategia,Exactitud en entrenamiento,Exactitud en validación,Exactitud en prueba
10,BernoulliNB,Eliminar filas,66.014670,65.693431,61.313869
0,GaussianNB,Eliminar filas,97.066015,94.160584,96.350365
6,MultinomialNB,Reemplazar con la media,89.498807,87.857143,90.000000


In [12]:
best_strategies = grid.loc[grid.groupby('Modelo')['Exactitud en prueba'].idxmax()]
best_strategies

,Modelo,Estrategia,Exactitud en entrenamiento,Exactitud en validación,Exactitud en prueba
11,BernoulliNB,Reemplazar con la media,66.348449,57.142857,71.428571
1,GaussianNB,Reemplazar con la media,96.897375,92.142857,97.142857
6,MultinomialNB,Reemplazar con la media,89.498807,87.857143,90.000000


Dado el dominio de los datos, es esperable que la distribución Gaussiana funcione mejor para este problema. 

Por otro lado es interesante que en la distribución Gaussiana la estrategia de eliminar filas funciona mejor en la validacion y que por otro lado la estrategia de rellenar con la media funciona mejor en la prueba.

In [13]:
from CustomNB import GaussianNaiveBayes
import numpy as np

In [14]:
cnb_data = macrodata[0].copy()
cnb_data = cnb_data.to_numpy()

X = cnb_data[:, :-1]
y = cnb_data[:, -1].astype(int)
y = np.where(y == 2, 0, 1)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

model = GaussianNaiveBayes()
model.fit(np.array(X_train), np.array(y_train), method='MLE')

y_train_pred_cnb = model.predict(np.array(X_train))
y_val_pred_cnb = model.predict(np.array(X_val))
y_test_pred_cnb = model.predict(np.array(X_test))

train_accuracy_cnb = accuracy_score(y_train, y_train_pred_cnb) * 100
val_accuracy_cnb = accuracy_score(y_val, y_val_pred_cnb) * 100
test_accuracy_cnb = accuracy_score(y_test, y_test_pred_cnb) * 100

print(f"CustomNB - Precisión en entrenamiento: {train_accuracy_cnb:.2f}%")
print(f"CustomNB - Precisión en validación: {val_accuracy_cnb:.2f}%")
print(f"CustomNB - Precisión en prueba: {test_accuracy_cnb:.2f}%")

CustomNB - Precisión en entrenamiento: 97.07%
CustomNB - Precisión en validación: 94.89%
CustomNB - Precisión en prueba: 96.35%
